# Introduction

Agents use LLMs to interact with other tools in the LangChain ecosystem and also come up with a step-by-step plan to complete your task.

We will be creating a basic agent in Langchain using the llama-2 model that can look for webpages (DuckDuckGo search) and YouTube videos (youtube_search) based on a prompt that I provide.

The notebook code is based on this tutorial video: https://www.youtube.com/watch?v=biS8G8x8DdA

In [ ]:
!pip -q install langchain
!pip -q install langchainhub
!pip -q install duckduckgo-search
!pip -q install --upgrade youtube_search
!pip -q install python-dotenv
!pip -q install ctransformers[cuda]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 105.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.6/412.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.7/867.7 kB 61.4 MB/s eta 0:00:00


# Import required libraries

Please ensure that you add an .env file with a HuggingFace token (HUGGINGFACEHUB_API_TOKEN environmenal variable) if you are using HuggingFace Hub to load your LLM

In [ ]:
from langchain.tools import DuckDuckGoSearchRun
from langchain.tools import YouTubeSearchTool
from langchain.llms import HuggingFaceHub
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from dotenv import load_dotenv
import sys

# Load HUGGINGFACEHUB_API_TOKEN environmenal variables (HuggingFace token) from .env file
# Uncomment if you use HuggingFace Hub to load your model
#load_dotenv()

False

First, test to make sure the DuckDuckGo search and YouTube search from Lancgchain works

In [ ]:
search = DuckDuckGoSearchRun()
print(search.run("FtsZ protein"))

yt = YouTubeSearchTool()
print(yt.run("Cat videos"))

FtsZ is the master cytomotive protein organizer of the divisome in most bacteria. It assembles into polymers at the division plane to form the 'Z ring' 1, which is required for the localization... FtsZ is a tubulin homolog GTPase protein that is widely conserved in bacteria and plays a key role in a complex called divisome, which carries out cell division 1, 2, 3. With GTP, FtsZ... The timing of cell division, and thus cell size in bacteria, is determined in part by the accumulation dynamics of the protein FtsZ, which forms the septal ring. FtsZ localization depends on... Filamenting temperature-sensitive mutant Z (FtsZ) is a crucial bacterial protein that forms a ring-like structure known as the Z-ring, playing a significant role in cell division. Targeting FtsZ is an effective approach for developing antibiotics that disrupt bacterial cell division and halt growth. FtsZ, a bacterial tubulin, plays a crucial role in cytokinesis process. Like tubulin, it also consists of two domains—N-

# LLM and tools

Here you will:

*  Define the LLM
*  Add the tools you defined above into a tool list. For each tool list, make sure you add a description of what the tool will be used for. The LLM will use the description you provide to select the best tool to answer your query

In [ ]:
from langchain.agents import Tool

tools = [
    Tool(
    name='DuckDuckGo Search',
    func= search.run,
    description="Useful for when you need to do a search websites to find information that another tool can't find. Be specific with your input."
    ),
    Tool(
    name='YoutTube Search',
    func= yt.run,
    description="Useful for when you need to do a search for videos. Be specific with your input."
    ),
]

## Define LLM

Use quantized Llama-13B (GGUF) with ctransformers as the LLM agent

In [ ]:
#from langchain.llms import LlamaCpp
from langchain_community.llms import CTransformers

# gpu_layers is meant to tell you how many layers to offload to GPU. Put a high number to load all of them to GPU
# context_length 4096 indicates max context length
config = {'max_new_tokens': 4096,
          'repetition_penalty': 1.1,
          'gpu_layers':50,
          'context_length' : 4096}

# Model file is the specific qunatization type that was done 2-bit, 3 bit etc... GGML_TYPE_Q2_K, GGML_TYPE_Q3_K etc...
# See example model files here: https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF#provided-files
llm = CTransformers(model="TheBloke/Llama-2-13B-chat-GGUF", model_file="llama-2-13b-chat.Q5_K_S.gguf", model_type="llama", config=config)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

llama-2-13b-chat.Q5_K_S.gguf:   0%|          | 0.00/8.97G [00:00<?, ?B/s]

In case you want to use Zephyr 7B or something else from HuggingFaceHub

In [ ]:
#repo_id = "HuggingFaceH4/zephyr-7b-beta"
#llm = HuggingFaceHub(repo_id=repo_id, model_kwargs={"temperature":0.2, "max_new_tokens":4000})

## Using the agents

In [ ]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

# Check prompt
print(prompt)

# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'


# Run chain

Try something simple like 'what is Langchain?'

In [ ]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               verbose=True,
                               max_iterations=3,
                               handle_parsing_errors=True)

agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...
 maybe search duckduckgo for "LangChain"
Action: DuckDuckGo Search
Action Input: "LangChain"LangChain is a framework for developing applications powered by language models, such as large language models (LLMs) and prompt engineering. Learn the basics of LangChain, its background, tech stack, and how to use it in Java with LangChain4j. Langchain is a tool that simplifies the creation of applications using large language models (LLMs) such as GPT-4. It provides a standard interface for chains, lots of integrations with other tools, and end-to-end chains for common applications. Learn how to install, use, and apply Langchain with Python and JavaScript. LangChain is a framework for creating applications that leverage the capabilities of language models. Learn how to install, set up, and use LangChain modules, such as Model I/O, Retrieval, Agents, Chains, Memory, and LangSmith. Explore common use cases and deploy with LangServe. LangChain is a Python 

{'input': 'what is LangChain?',
 'output': 'The final answer to the question "what is LangChain" is that it is a framework for developing applications powered by language models, such as large language models (LLMs) and prompt engineering. It provides a standard interface for chains, lots of integrations with other tools, and end-to-end chains for common applications.'}

Try getting cat videos from YouTube

In [ ]:
agent_executor.invoke({"input": "Give me two links to YouTube videos of cute cats"})



> Entering new AgentExecutor chain...
 Hmm, I wonder if there are any websites that have a collection of funny cat videos.
Action: YoutTube Search
Action Input: "funny cat videos"['https://www.youtube.com/watch?v=OX10yEOsOWE&pp=ygUQZnVubnkgY2F0IHZpZGVvcw%3D%3D', 'https://www.youtube.com/watch?v=CoqYVImPwuE&pp=ygUQZnVubnkgY2F0IHZpZGVvcw%3D%3D'] These look like great videos, but I want to see more!
Action: YoutTube Search
Action Input: "funny cats compilations"['https://www.youtube.com/watch?v=JxS5E-kZc2s&pp=ygUXZnVubnkgY2F0cyBjb21waWxhdGlvbnM%3D', 'https://www.youtube.com/watch?v=N8siuNjyV7A&pp=ygUXZnVubnkgY2F0cyBjb21waWxhdGlvbnM%3D'] These are great, but I want to see more variety!
Action: YoutTube Search
Action Input: "funny cat videos different breeds"['https://www.youtube.com/watch?v=eEduXbd7sMI&pp=ygUhZnVubnkgY2F0IHZpZGVvcyBkaWZmZXJlbnQgYnJlZWRz', 'https://www.youtube.com/watch?v=ywSFaeSEQIU&pp=ygUhZnVubnkgY2F0IHZpZGVvcyBkaWZmZXJlbnQgYnJlZWRz']

> Finished chain.


{'input': 'Give me two links to YouTube videos of cute cats',
 'output': 'Agent stopped due to iteration limit or time limit.'}

What about something something scientific related to a cell division protein called FtsZ?

In [ ]:
agent_executor.invoke({"input": "What is FtsZ and what are some videos related to FtsZ?"})



> Entering new AgentExecutor chain...
 Hmm, I'm not sure what FtsZ is. Let me search for it on YouTube.
Action: YoutTube Search
Action Input: FtsZ['https://www.youtube.com/watch?v=SQ4nhZnONl8&pp=ygUERnRzWg%3D%3D', 'https://www.youtube.com/watch?v=IzKoIp08Pa4&pp=ygUERnRzWg%3D%3D'] Hmm, there are several videos about FtsZ. Let me check the first one.
Action: YoutTube Search
Action Input: SQ4nhZnONl8['https://www.youtube.com/watch?v=SQ4nhZnONl8&pp=ygULU1E0bmhabk9ObDg%3D', 'https://www.youtube.com/shorts/bcNzAFtwZ7E'] These videos are talking about FtsZ as a protein that is essential for bacterial cell division. It seems to be an important topic in microbiology.
Action: DuckDuckGo Search
Action Input: FtsZ bacteria microbiology importanceFtsZ is the master cytomotive protein organizer of the divisome in most bacteria. It assembles into polymers at the division plane to form the 'Z ring' 1, which is required for the localization... The timing of cell division, and thus cell size in bacter

{'input': 'What is FtsZ and what are some videos related to FtsZ?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

You can see that it has used both DuckDuckGo search and YouTube to complete my query